In [ ]:
import pulsestreamer as ps
import numpy as np

In [ ]:
ip = '169.254.8.2'
pulse_streamer = ps.PulseStreamer(ip)
pulse_streamer.selectClock(ps.ClockSource.INTERNAL)

Connect to Pulse Streamer via JSON-RPC.
IP / Hostname: 169.254.8.2
Pulse Streamer 8/2 firmware: v1.7.2
Client software: v1.7.0


### Convert Pi3 format to PulseStreamer format

In [33]:
# Typical Rabi sequence in Pi3diamond format
channel_map = {
    'aom':0, 
    'detect':1,
    'sync':2,
    'mw':3,
}

sync_time   = 100.
wait        = 1000.
laser_time  = 1000.
tau_start   = 0.
tau_delta   = 100.
num_inc     = 10.
tau_end     = tau_start + num_inc*tau_delta + tau_delta/2.
tau_list = np.arange(tau_start, tau_end, num_inc)

sequence = [(['sync'], sync_time)]
for t in tau_list:
    sequence += [([],wait), (['mw'],t), ([],tau_end - t), (['aom','detect'], laser_time)]

for p in sequence[:10]: print(p)

(['sync'], 100.0)
([], 1000.0)
(['mw'], 0.0)
([], 1050.0)
(['aom', 'detect'], 1000.0)
([], 1000.0)
(['mw'], 10.0)
([], 1040.0)
(['aom', 'detect'], 1000.0)
([], 1000.0)


In [34]:
# Convert it to pulsestreamer format
pattens_by_channels = dict()
for ch_name in channel_map.keys(): pattens_by_channels[ch_name] = []

time_stamp = 0
min_timestep = 1.
for ch_high, duration in sequence:
    if duration < min_timestep: continue
    for ch_name, ch in channel_map.items():
        output = 1 if ch_name in ch_high else 0
        pattens_by_channels[ch_name].append((duration, output))

In [ ]:
seq = pulse_streamer.createSequence()

for ch_name, patt in pattens_by_channels.items():
    print("%d %6s"%(channel_map[ch_name], ch_name), patt[:10])
    seq.setDigital(channel_map[ch_name], patt)

In [ ]:
pulse_streamer.stream(seq)  # Start

In [ ]:
pulse_streamer.reset() # Stop

### Simple demo on how to create, start and stop a pulse sequence

In [10]:
pulse_patt1 = [(100, 0), (200, 1), (80, 0), (300, 1), (10140, 0)]
pulse_patt2 = [(120, 0), (220, 1), (100, 0), (320, 1), (10060, 0)]
seq = pulse_streamer.createSequence()
seq.setDigital(1, pulse_patt1)
seq.setDigital(2, pulse_patt2)
pulse_streamer.stream(seq)

In [11]:
pulse_streamer.reset()

0